# Simulazio kutxaren irudiak sortzeko beharrezko pdb-en sorrera

In [15]:
import numpy as np
import pandas as pd
from copy import deepcopy
import matplotlib.pyplot as plt
import openmm as mm
from openmm import app, unit, Vec3
from openmmtools.constants import ONE_4PI_EPS0
from openmm.unit.quantity import Quantity
from openmmforcefields.generators import GAFFTemplateGenerator
from openff.toolkit.topology import Molecule

In [11]:
molecule = Molecule('ibuprofeno.sdf')
molecule_pdb = app.PDBFile('ibuprofeno.pdb')

#molecule = Molecule('molecule.sdf')
#molecule_pdb = app.PDBFile('molecule.pdb')

In [12]:
forcefield = app.ForceField('amber14/tip3pfb.xml')
gaff = GAFFTemplateGenerator(molecule, forcefield='gaff-2.11')
forcefield.registerTemplateGenerator(gaff.generator)

solvated_model = app.Modeller(molecule_pdb.topology, molecule_pdb.positions)
solvated_model.addSolvent(forcefield, model='tip3p', padding=1.5 * unit.nanometer)

system = forcefield.createSystem(
    topology=solvated_model.topology,
    nonbondedMethod=app.CutoffPeriodic,
    nonbondedCutoff=0.8 * unit.nanometer,
    constraints=app.HBonds,
    rigidWater=True
)

In [16]:
# Hutseko sistema prestatu:
vacuum_model = app.Modeller(molecule_pdb.topology, molecule_pdb.positions)
vacuum_model.topology.setPeriodicBoxVectors(Quantity(value=(
    Vec3(x=3.0, y=0.0, z=0.0),
    Vec3(x=0.0, y=3.0, z=0.0),
    Vec3(x=0.0, y=0.0, z=3.0)),
    unit = unit.nanometer
))

vacuum_system = forcefield.createSystem(
    topology = vacuum_model.topology,
    nonbondedMethod = app.NoCutoff, # Hau da aldaketa nagusia! Cutoffak eta PBCak kentzeko.
    # Remove: nonbondedCutoff = ..., 
    removeCMMotion = True,
    constraints = app.HBonds,
    rigidWater = True
)

In [18]:
temperature = 300 * unit.kelvin
friction_coefficient = 1 / unit.picosecond
time_step = 4 * unit.femtosecond

integrator = mm.LangevinMiddleIntegrator(temperature, friction_coefficient,time_step)
integrator_vacuum = mm.LangevinMiddleIntegrator(temperature, friction_coefficient,time_step)

In [19]:
# Simulazioari esan behar diogu zein hardware erabili behar duen simulazioa egiteko. 
# Hemen 'CPU' jarri txartel grafikorik ez badago ordenagailuan.
platform = mm.Platform.getPlatformByName('CUDA')

simulation = app.Simulation(solvated_model.topology, system, integrator, platform=platform)
simulation.context.setPositions(solvated_model.positions)

simulation_vacuum = app.Simulation(vacuum_model.topology, vacuum_system, integrator_vacuum, platform=platform)
simulation_vacuum.context.setPositions(vacuum_model.positions)

In [24]:
pdb_reporter = app.PDBReporter(file='Water_system.pdb',reportInterval=1)
pdb_reporter_vacuum = app.PDBReporter(file='Vacuum_system.pdb',reportInterval=1)
simulation.reporters.append(pdb_reporter)
simulation_vacuum.reporters.append(pdb_reporter_vacuum)


In [25]:
simulation.step(1)
simulation_vacuum.step(1)